#### This notebook demonstrates the use of adversarial debiasing algorithm to learn a fair classifier.
Adversarial debiasing [1] is an in-processing technique that learns a classifier to maximize prediction accuracy and simultaneously reduce an adversary's ability to determine the protected attribute from the predictions. This approach leads to a fair classifier as the predictions cannot carry any group discrimination information that the adversary can exploit. We will see how to use this algorithm for learning models with and without fairness constraints and apply them on the Adult dataset.

In [6]:
# Install AIF360
!pip install aif360

In [7]:
%matplotlib inline
# Load all necessary packages
import sys
sys.path.append("../")
from aif360.datasets import BinaryLabelDataset
from aif360.datasets import AdultDataset, GermanDataset, CompasDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult, load_preproc_data_compas, load_preproc_data_german

from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt

import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

#### Load dataset and set options

In [10]:
import urllib.request
import os
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult

# URLs for the dataset files
urls = [
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test",
    "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names"
]

# Folder to store the dataset files
data_folder = "/usr/local/lib/python3.10/dist-packages/aif360/data/raw/adult/"

# Ensure the folder exists
os.makedirs(data_folder, exist_ok=True)

# Download and save the dataset files
for url in urls:
    file_name = url.split("/")[-1]
    file_path = os.path.join(data_folder, file_name)
    urllib.request.urlretrieve(url, file_path)
    print(f"Downloaded {file_name} and saved to {file_path}")

# Load the dataset into the dataset_orig variable
dataset_orig = load_preproc_data_adult()


Downloaded adult.data and saved to /usr/local/lib/python3.10/dist-packages/aif360/data/raw/adult/adult.data
Downloaded adult.test and saved to /usr/local/lib/python3.10/dist-packages/aif360/data/raw/adult/adult.test
Downloaded adult.names and saved to /usr/local/lib/python3.10/dist-packages/aif360/data/raw/adult/adult.names


In [11]:
# Get the dataset and split into train and test
privileged_groups = [{'sex': 1}]
unprivileged_groups = [{'sex': 0}]

dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

In [12]:
# print out some labels, names, etc.
display(Markdown("#### Training Dataset shape"))
print(dataset_orig_train.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_orig_train.favorable_label, dataset_orig_train.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_orig_train.protected_attribute_names)
display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_orig_train.privileged_protected_attributes,
      dataset_orig_train.unprivileged_protected_attributes)
display(Markdown("#### Dataset feature names"))
print(dataset_orig_train.feature_names)

#### Training Dataset shape

(34189, 18)


#### Favorable and unfavorable labels

1.0 0.0


#### Protected attribute names

['sex', 'race']


#### Privileged and unprivileged protected attribute values

[array([1.]), array([1.])] [array([0.]), array([0.])]


#### Dataset feature names

['race', 'sex', 'Age (decade)=10', 'Age (decade)=20', 'Age (decade)=30', 'Age (decade)=40', 'Age (decade)=50', 'Age (decade)=60', 'Age (decade)=>=70', 'Education Years=6', 'Education Years=7', 'Education Years=8', 'Education Years=9', 'Education Years=10', 'Education Years=11', 'Education Years=12', 'Education Years=<6', 'Education Years=>12']


#### Metric for original training data

In [13]:
# Metric for the original dataset
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train,
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())
metric_orig_test = BinaryLabelDatasetMetric(dataset_orig_test,
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_test.mean_difference())

#### Original training dataset

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.201473
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.178281


In [14]:
min_max_scaler = MaxAbsScaler()
dataset_orig_train.features = min_max_scaler.fit_transform(dataset_orig_train.features)
dataset_orig_test.features = min_max_scaler.transform(dataset_orig_test.features)
metric_scaled_train = BinaryLabelDatasetMetric(dataset_orig_train,
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
display(Markdown("#### Scaled dataset - Verify that the scaling does not affect the group label statistics"))
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_scaled_train.mean_difference())
metric_scaled_test = BinaryLabelDatasetMetric(dataset_orig_test,
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_scaled_test.mean_difference())


#### Scaled dataset - Verify that the scaling does not affect the group label statistics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.201473
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.178281


### Learn plan classifier without debiasing

In [15]:
# Load post-processing algorithm that equalizes the odds
# Learn parameters with debias set to False
sess = tf.Session()
plain_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='plain_classifier',
                          debias=False,
                          sess=sess)

In [16]:
plain_model.fit(dataset_orig_train)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


epoch 0; iter: 0; batch classifier loss: 0.728883
epoch 0; iter: 200; batch classifier loss: 0.394524
epoch 1; iter: 0; batch classifier loss: 0.480390
epoch 1; iter: 200; batch classifier loss: 0.380544
epoch 2; iter: 0; batch classifier loss: 0.394231
epoch 2; iter: 200; batch classifier loss: 0.501938
epoch 3; iter: 0; batch classifier loss: 0.464210
epoch 3; iter: 200; batch classifier loss: 0.393823
epoch 4; iter: 0; batch classifier loss: 0.371760
epoch 4; iter: 200; batch classifier loss: 0.434270
epoch 5; iter: 0; batch classifier loss: 0.445593
epoch 5; iter: 200; batch classifier loss: 0.386621
epoch 6; iter: 0; batch classifier loss: 0.434529
epoch 6; iter: 200; batch classifier loss: 0.401859
epoch 7; iter: 0; batch classifier loss: 0.307391
epoch 7; iter: 200; batch classifier loss: 0.427325
epoch 8; iter: 0; batch classifier loss: 0.481979
epoch 8; iter: 200; batch classifier loss: 0.419054
epoch 9; iter: 0; batch classifier loss: 0.427229
epoch 9; iter: 200; batch classi

In [17]:
# Apply the plain model to test data
dataset_nodebiasing_train = plain_model.predict(dataset_orig_train)
dataset_nodebiasing_test = plain_model.predict(dataset_orig_test)

In [18]:
# Metrics for the dataset from plain model (without debiasing)
display(Markdown("#### Plain model - without debiasing - dataset metrics"))
metric_dataset_nodebiasing_train = BinaryLabelDatasetMetric(dataset_nodebiasing_train,
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_train.mean_difference())

metric_dataset_nodebiasing_test = BinaryLabelDatasetMetric(dataset_nodebiasing_test,
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_test.mean_difference())

display(Markdown("#### Plain model - without debiasing - classification metrics"))
classified_metric_nodebiasing_test = ClassificationMetric(dataset_orig_test,
                                                 dataset_nodebiasing_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_nodebiasing_test.accuracy())
TPR = classified_metric_nodebiasing_test.true_positive_rate()
TNR = classified_metric_nodebiasing_test.true_negative_rate()
bal_acc_nodebiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_nodebiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_nodebiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_nodebiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_nodebiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_nodebiasing_test.theil_index())

#### Plain model - without debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.207929
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.212740


#### Plain model - without debiasing - classification metrics

Test set: Classification accuracy = 0.803726
Test set: Balanced classification accuracy = 0.659056
Test set: Disparate impact = 0.000000
Test set: Equal opportunity difference = -0.460873
Test set: Average odds difference = -0.284938
Test set: Theil_index = 0.176748


### Apply in-processing algorithm based on adversarial learning

In [19]:
sess.close()
tf.reset_default_graph()
sess = tf.Session()

In [20]:
# Learn parameters with debias set to True
debiased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='debiased_classifier',
                          debias=True,
                          sess=sess)

In [21]:
debiased_model.fit(dataset_orig_train)

epoch 0; iter: 0; batch classifier loss: 0.748581; batch adversarial loss: 0.661708
epoch 0; iter: 200; batch classifier loss: 0.518333; batch adversarial loss: 0.668401
epoch 1; iter: 0; batch classifier loss: 0.413743; batch adversarial loss: 0.637986
epoch 1; iter: 200; batch classifier loss: 0.429247; batch adversarial loss: 0.648797
epoch 2; iter: 0; batch classifier loss: 0.476008; batch adversarial loss: 0.643821
epoch 2; iter: 200; batch classifier loss: 0.416968; batch adversarial loss: 0.663498
epoch 3; iter: 0; batch classifier loss: 0.461186; batch adversarial loss: 0.637949
epoch 3; iter: 200; batch classifier loss: 0.498391; batch adversarial loss: 0.678092
epoch 4; iter: 0; batch classifier loss: 0.454445; batch adversarial loss: 0.608626
epoch 4; iter: 200; batch classifier loss: 0.433467; batch adversarial loss: 0.633072
epoch 5; iter: 0; batch classifier loss: 0.439382; batch adversarial loss: 0.650255
epoch 5; iter: 200; batch classifier loss: 0.405386; batch adversa

In [22]:
# Apply the plain model to test data
dataset_debiasing_train = debiased_model.predict(dataset_orig_train)
dataset_debiasing_test = debiased_model.predict(dataset_orig_test)

In [23]:
# Metrics for the dataset from plain model (without debiasing)
display(Markdown("#### Plain model - without debiasing - dataset metrics"))
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_train.mean_difference())
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_nodebiasing_test.mean_difference())

# Metrics for the dataset from model with debiasing
display(Markdown("#### Model - with debiasing - dataset metrics"))
metric_dataset_debiasing_train = BinaryLabelDatasetMetric(dataset_debiasing_train,
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_debiasing_train.mean_difference())

metric_dataset_debiasing_test = BinaryLabelDatasetMetric(dataset_debiasing_test,
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_dataset_debiasing_test.mean_difference())



display(Markdown("#### Plain model - without debiasing - classification metrics"))
print("Test set: Classification accuracy = %f" % classified_metric_nodebiasing_test.accuracy())
TPR = classified_metric_nodebiasing_test.true_positive_rate()
TNR = classified_metric_nodebiasing_test.true_negative_rate()
bal_acc_nodebiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_nodebiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_nodebiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_nodebiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_nodebiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_nodebiasing_test.theil_index())



display(Markdown("#### Model - with debiasing - classification metrics"))
classified_metric_debiasing_test = ClassificationMetric(dataset_orig_test,
                                                 dataset_debiasing_test,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % classified_metric_debiasing_test.accuracy())
TPR = classified_metric_debiasing_test.true_positive_rate()
TNR = classified_metric_debiasing_test.true_negative_rate()
bal_acc_debiasing_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_debiasing_test)
print("Test set: Disparate impact = %f" % classified_metric_debiasing_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % classified_metric_debiasing_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % classified_metric_debiasing_test.average_odds_difference())
print("Test set: Theil_index = %f" % classified_metric_debiasing_test.theil_index())

#### Plain model - without debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.207929
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.212740


#### Model - with debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.086742
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.087589


#### Plain model - without debiasing - classification metrics

Test set: Classification accuracy = 0.803726
Test set: Balanced classification accuracy = 0.659056
Test set: Disparate impact = 0.000000
Test set: Equal opportunity difference = -0.460873
Test set: Average odds difference = -0.284938
Test set: Theil_index = 0.176748


#### Model - with debiasing - classification metrics

Test set: Classification accuracy = 0.795605
Test set: Balanced classification accuracy = 0.675668
Test set: Disparate impact = 0.581453
Test set: Equal opportunity difference = -0.044364
Test set: Average odds difference = -0.033521
Test set: Theil_index = 0.166363



    References:
    [1] B. H. Zhang, B. Lemoine, and M. Mitchell, "Mitigating UnwantedBiases with Adversarial Learning,"
    AAAI/ACM Conference on Artificial Intelligence, Ethics, and Society, 2018.